# Обучение моделей в Pytorch

## Dataset and DataLoader

In [ ]:
import torch
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(train_dataset, batch_size=64, num_workers=0, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=0, shuffle=True)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [ ]:
def show_fashion_plots(
    dataset: Dataset, labels_map: dict[int, str], cols: int = 3, rows: int = 3
) -> None:
    figure = plt.figure(figsize=(8, 8))
    for i in range(1, cols * rows + 1):
        sample_idx = torch.randint(len(dataset), size=(1,)).item()
        img, label = dataset[sample_idx]
        figure.add_subplot(rows, cols, i)
        if type(label) == torch.Tensor:
            plt.title(labels_map[label.item()])
        else:
            plt.title(labels_map[label])
        plt.axis("off")
        plt.imshow(img.squeeze(), cmap="gray")

    plt.show()

In [ ]:
show_fashion_plots(train_dataset, labels_map=labels_map)

## NN module

Нейронные сети состоят из слоев/модулей, которые выполняют операции с данными. Пакет torch.nn предоставляет все строительные блоки, необходимые для создания собственной нейронной сети. Каждый модуль в pytorch является подклассом nn.Module. Нейронная сеть — это сам модуль, состоящий из других модулей (слоев).

In [ ]:
from torch import nn
from torchvision import datasets, transforms


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

Проиллюстрируем на примере ~многослойной нейронной сети прямой связи~ feed-forward нейронной сети (название многослойный персептрон используется чаще, но за персептроном зафиксирован конкретный тип модели, разработанный в середине прошлого века).

Важно отметить, что каждая составная часть модели представляет собой наследника nn.Module.

In [ ]:
class FFNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.flatten = nn.Flatten()  # векторизация изображения-матрицы
        self.linear_relu_stack = nn.Sequential(  # контейнер для модулей
            nn.Linear(  # линейная трансформация, bias=True on default
                in_features=28 * 28, out_features=512, bias=True
            ),
            nn.ReLU(),  # нелинейная функция активации
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(  # основной метод, связывающий инициализированные слои в вычислительный граф
        self, x: torch.LongTensor
    ) -> torch.FloatTensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
ffnn_model = FFNN().to(DEVICE)  # переносим веса модели на ГПУ (при наличии)
print(ffnn_model)

Ниже пример работы модели и трансформации ее вывода

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
logits = ffnn_model(
    train_features.to(DEVICE)
)  # "raw" предсказания модели, [-\inf, \inf]
pred_probas = nn.Softmax(dim=1)(
    logits  # логиты передаем в софтмакс, трансформирующий значения в интервал [0; 1]
)
print("Пример вывода модели для одного сэмпла данных:", pred_probas[0], sep="\n")
print("Размерность вывода:", pred_probas.shape)
y_pred = pred_probas.argmax(
    1
)  # наконец, выбираем наиболее вероятный класс для каждого сэмпла
y_pred

In [ ]:
# вывод всех параметров модели
for name, param in ffnn_model.named_parameters():
    print(
        f"Layer: {name}",
        f"Size: {param.size()}",
        f"Values : {param[:2]}",
        sep="\n",
        end="\n\n",
    )

In [ ]:
def count_model_params(model: nn.Module) -> int:
    """Returns the amout of pytorch model parameters."""
    return sum(p.numel() for p in model.parameters())

In [ ]:
count_model_params(ffnn_model)

Рассмотрим альтернативный (stateless) способ создания моделей и описания ее работы в методе forward: functional API.

In [ ]:
from torch.nn import functional as F

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes: int = 10, dropout_p: float = 0.1) -> None:
        super().__init__()

        self.dropout_p = dropout_p

        # инстанцируем лишь модули с обучаемыми параметрами (и объекты nn.Parameter)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(F.dropout1d(x, p=self.dropout_p)))
        x = self.fc2(F.dropout1d(x, p=self.dropout_p))
        return x

А вот, как это выглядело бы в Module API:

In [ ]:
class CNN_M(nn.Module):
    def __init__(self, num_classes: int = 10, dropout_p: float = 0.1) -> None:
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_p),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_p),
            nn.Linear(128, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
cnn_model = CNN().to(DEVICE)
print(cnn_model)

In [ ]:
logits = cnn_model(train_features.to(DEVICE))
pred_probas = nn.Softmax(dim=1)(logits)
y_pred = pred_probas.argmax(1)
y_pred

In [ ]:
count_model_params(cnn_model)

Проверим, что количество параметров у модульной версии то же самое

In [ ]:
cnn_model = CNN_M().to(DEVICE)
print(cnn_model)
count_model_params(cnn_model)

Преимущество функционального способа понятно: больший простор для связывания отдельных блоков. Недостаток очевиден: читаемость кода ухудшается. Идеального рецепта не существует, всегда нужно искать баланс.

## Оптимизатор

Оптимизатор ниже (стохастический градиентный спуск) -- алгоритм настройки параметров (весов) нейронной сети, который на основе данных стремится минимизировать значение лосс-функции.

Оптимизаторов немало, вас могли были познакомить с некоторыми из них на курсе по методам оптимизации. Полный список реализованных в pytorch можно посмотреть тут: https://pytorch.org/docs/stable/optim.html

На вход оптимизатору подаем параметры модели и все необходимые гиперпараметры алгоритма оптимизации. Совсем необязательно настраивать все параметры, часть из них может быть "заморожена", но об этом поговорим на следующих занятиях.

In [ ]:
optimizer = torch.optim.SGD(ffnn_model.parameters(), lr=1e-3)

## Model training

In [ ]:
LEARNING_RATE = 1e-1
BATCH_SIZE = 128
EPOCHS_NUM = 10

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, num_workers=0, shuffle=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, num_workers=0, shuffle=True
)

In [ ]:
ffnn_model = FFNN().to(DEVICE)
cnn_model = CNN().to(DEVICE)
model = cnn_model

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [ ]:
from typing import Callable


def train_loop(
    dataloader: DataLoader,
    model: nn.Module,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    device: str,
) -> None:
    model.train()

    size = len(dataloader.dataset)
    batches_num = len(dataloader)
    train_loss, correct = 0.0, 0
    for batch, data in enumerate(dataloader):
        inputs, targets = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        preds = model(inputs)
        loss = loss_fn(preds, targets)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (preds.argmax(1) == targets).type(torch.float).sum().item()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * BATCH_SIZE + len(inputs)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= batches_num
    correct /= size
    print(
        f"Train Error: Accuracy: {(100 * correct):>0.1f}%, Avg loss: {train_loss:>8f} \n"
    )


def test_loop(
    dataloader: DataLoader,
    model: nn.Module,
    loss_fn: Callable,
    device: str,
) -> None:
    model.eval()

    size = len(dataloader.dataset)
    batches_num = len(dataloader)
    test_loss, correct = 0.0, 0
    with torch.no_grad():
        for data in dataloader:
            inputs, targets = data[0].to(device), data[1].to(device)

            preds = model(inputs)
            test_loss += loss_fn(preds, targets).item()
            correct += (preds.argmax(1) == targets).type(torch.float).sum().item()

    test_loss /= batches_num
    correct /= size
    print(
        f"Test Error: Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [ ]:
loss_fn = nn.CrossEntropyLoss()

for epoch in range(EPOCHS_NUM):
    print(f"Epoch {epoch + 1}")
    train_loop(
        dataloader=train_dataloader,
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        device=DEVICE,
    )
    test_loop(
        dataloader=test_dataloader,
        model=model,
        loss_fn=loss_fn,
        device=DEVICE,
    )
    print("-" * 15)

## Сохранение и загрузка модели

In [ ]:
model_path = f"{model.__class__.__name__.lower()}.pt"
torch.save(
    {"misc": "misc", "model_state_dict": model.state_dict()},
    model_path,
)

In [ ]:
model_state_dict = torch.load(model_path, map_location="cpu")["model_state_dict"]
model_loaded = model.__class__()
model_loaded.load_state_dict(model_state_dict)
model_loaded.to(DEVICE)

## Inference

In [ ]:
sample_idx = 124
img, label = test_dataset[sample_idx]

plt.imshow(img.squeeze(0))
plt.title(labels_map[label])

In [ ]:
preds = model_loaded(img.to(DEVICE))
print("Предсказание модели: ", labels_map[preds.argmax(1).item()])

## Дополнительные топики

* Сохранение лучшей модели: Checkpointer
* Изменение скорости обучения: torch.optim.lr_scheduler
* Модели компьютерного зрения в пакете torchvision: https://github.com/pytorch/vision/tree/main/torchvision/models
* Фреймворк глубокого обучения для упрощения процесса обучения моделей глубокого обучения (переусложнение преднамеренное): https://github.com/Lightning-AI/pytorch-lightning (сайт заблокирован для ru-зоны)
* Huggingface's Accelerate: https://huggingface.co/docs/accelerate/index
* инструменты для разметки датасетов
* deploy

## Задание на оставшееся время

Начните писать собственный кастомный класс Dataset для своей задачи: будь то проект для ВКР или любая другая идея, которую вы хотите протестировать с помощью глубокого обучения. Если нет идей, начните с анализа кода готовых датасетов в библиотеке torchvision: https://pytorch.org/vision/0.16/datasets.html